## Team member: Chao Wan(A20345781), Dalin Yang(A20358102)

Import library and read the data from file.

In [4]:
import numpy as np  
import scipy as sp  
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve  
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.mlab as mlab    
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import SGDClassifier

In [5]:
f = open("communities-crime-clean.csv", "r")

data = []
crimeRate = []
highCrime = []
features = f.readline().strip().split(",")
line = f.readline()
while line:
    tokens = line.strip().split(",")
    tuple = []
    for attr in tokens[3:-1]:
        tuple.append(float(attr))
    data.append(tuple)
    crimeRate.append(float(tokens[-1]))
    if(float(tokens[-1]) > 0.1):
        highCrime.append(True)
    else:
        highCrime.append(False)
    line = f.readline()
x = np.array(data)
y = np.array(highCrime)
y2= np.array(crimeRate)
f.close()

## 1. Decision Tree

### a. Postive Percentage & Nagetive Percentage

In [3]:
PositivePercent = np.sum(y==True)/len(y)
PositivePercent

0.62719518314099343

In [4]:
NagetivePercent = np.sum(y==False)/len(y)
NagetivePercent

0.37280481685900652

### b. Use DecisionTreeClassifier

In [47]:
clf = tree.DecisionTreeClassifier()    
clf.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

#### i. Accuracy, Precision, Recall

In [6]:
accuracy = accuracy_score(y, y)
print("Accuracy:", accuracy, "\n")
answer = clf.predict_proba(x)[:,1]  
print(classification_report(y, answer, target_names = ['LowCrime', 'HighCrime'])) 

Accuracy: 1.0 

             precision    recall  f1-score   support

   LowCrime       1.00      1.00      1.00       743
  HighCrime       1.00      1.00      1.00      1250

avg / total       1.00      1.00      1.00      1993



#### ii.Mean Features:

The mean featrues are PctKids2Par, racePctWhite and racePctHisp. 
Because when we tracing the data, we will find the PctKidsPar or racePctWhite or racePctHisp with higher number will have lower crime rate, and the vice versa. 

In [59]:
clf = tree.DecisionTreeClassifier()    
clf.fit(x, y)
nvs = zip(features[3:-1], clf.feature_importances_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('PctKids2Par', 0.3597251889048339),
 ('racePctWhite', 0.090533571999860493),
 ('racePctHisp', 0.050532236222209419),
 ('PctLess9thGrade', 0.022780774918367497),
 ('PctEmplManu', 0.017035270197975357),
 ('MedRent', 0.015301814701351533),
 ('PctImmigRec10', 0.014596466282100392),
 ('PctHousOwnOcc', 0.013685604733908239),
 ('PctSpeakEnglOnly', 0.012476414096854615),
 ('PctSameState85', 0.012219169785146998)]

### c. Apply cross-validation

#### i. Accuracy, precision, recall

In [48]:
answer = cross_val_score(clf, x, y, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.75        0.795       0.745       0.74371859  0.67336683  0.64824121
  0.72361809  0.68844221  0.77889447  0.73366834]
The average of accuracy: 0.727994974874


In [49]:
answer = cross_val_score(clf, x, y, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.77697842  0.83206107  0.76335878  0.78518519  0.77884615  0.73584906
  0.77777778  0.86021505  0.76223776  0.81896552]
The average of precision: 0.789147477125


In [50]:
answer = cross_val_score(clf, x, y, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.856  0.896  0.76   0.872  0.608  0.664  0.8    0.656  0.896  0.744]
The average of recall: 0.7752


#### ii. The Reason of Difference

Since the decision tree is only based on the training data, it will got mistake when applying to the test data which should be different with trainning data. Therefore, the result will not be 100% match.

Howerver, the previous decision tree is trained on the entire dataset, and also test on these data, the result should obvious be the 100% match. It's kind of overfitting.

## 2. Linear Classification

### a. Use GaussianNB

#### i.Accuracy, precision and recall

In [55]:
clf = GaussianNB()
answer = cross_val_score(clf, x, y, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.775       0.8         0.825       0.79899497  0.70351759  0.65326633
  0.81407035  0.73366834  0.71356784  0.79899497]
The average of accuracy: 0.761608040201


In [52]:
answer = cross_val_score(clf, x, y, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.86363636  0.92929293  0.95        0.92079208  0.94594595  0.86842105
  0.92307692  1.          0.77868852  0.93814433]
The average of precision: 0.911799814828


In [53]:
answer = cross_val_score(clf, x, y, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.76   0.736  0.76   0.744  0.56   0.528  0.768  0.576  0.76   0.728]
The average of recall: 0.692


#### ii. Most Predictive Features

In [49]:
clf = GaussianNB()
answer = clf.fit(x, y)
diff = abs(answer.theta_[0] - answer.theta_[1])/(answer.sigma_[0] + answer.sigma_[1])
nvs = zip(features[3:-1], diff)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('PctKids2Par', 5.0013102290317129),
 ('FemalePctDiv', 4.7568859628347688),
 ('PctFam2Par', 4.5939880055148983),
 ('pctWInvInc', 4.3949908462647498),
 ('TotalPctDiv', 4.3859860760484883),
 ('PctTeen2Par', 3.9993536543892376),
 ('MalePctDivorce', 3.9507314134882674),
 ('PctYoungKids2Par', 3.6483010761779187),
 ('PctIlleg', 3.4698885121965644),
 ('racePctWhite', 3.4432283131853589)]

It make sence. If the feature has higher rank, it means that the difference of means of this feature bewteen two classes is relative big or the varience is raletive small. Therefore, this kind feature has a clearer classification between two classes since it has a relative clear gap, and so it can be used to predict output (classes). Additionally, in the file, these features has the obviously seperated output (classes) along the ascendent feature values.

#### iii. Compare to decision tree

The GaussianNB has much better precision but worse recall, and the precision is little better. This shows that the GaussianNB may not find most positive data, but the data has a high possibility to be positive when GaussianNB predict it as positive.

For key features, few features both have similar ranks and majority features got different ranks in GaussianNB, for example, the racePctWhite. This is because although they has a better thesthold to seperate the two classes, the means of these two classes may be close or the data in two classes spread too widely.

### b. LinearSVC

#### i. 10-fold cross-validation

In [56]:
clf = LinearSVC()
answer = cross_val_score(clf, x, y, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.79        0.865       0.84        0.80904523  0.69346734  0.69346734
  0.81407035  0.8241206   0.81909548  0.81407035]
The average of accuracy: 0.796233668342


In [59]:
answer = cross_val_score(clf, x, y, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.76774194  0.84507042  0.82517483  0.85950413  0.87209302  0.7962963
  0.84920635  0.95918367  0.80689655  0.87288136]
The average of precision: 0.845404856531


In [58]:
answer = cross_val_score(clf, x, y, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.952  0.96   0.944  0.832  0.6    0.688  0.856  0.752  0.936  0.824]
The average of recall: 0.8344


#### ii. 10 most predict feature

If we just see the signle features, some of them do not make sence. But if we caculate them with weight together, it can seperate the two classes clearly, since this method take account all the features to predict highCrime.

In [50]:
clf = LinearSVC()
answer = clf.fit(x,y)
nvs = zip(features[3:-1], abs(answer.coef_[0]))
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('pctWInvInc', 1.8884872753574939),
 ('PersPerOccupHous', 1.7551236098625982),
 ('racePctWhite', 1.5002170472255181),
 ('PctKids2Par', 1.1903278739440728),
 ('RentHighQ', 1.0668837880418409),
 ('MalePctDivorce', 1.0656961744329736),
 ('NumUnderPov', 1.0515482755155021),
 ('NumStreet', 1.019143992337431),
 ('PctOccupMgmtProf', 1.0146749260525534),
 ('population', 1.0023012880707545)]

#### iii. Compare to decision tree

All the CV results become better, since the Linear SVC consider the relationship between features while decision tree does not care it.

For key featrues, also there is a big difference between the results of LinearSVC and Decision tree. The reason is that LinearSVC is trying to find a hyperplane to classify and take all the features into account, so the weight of one feature is affects by the distribution of other featrues. However, the decision tree just consider one featrue to find the most important feature in one step.   

## 3. Regression

### a. Use LinearRegression

#### i. 10-fold cross-validation

In [14]:
LR_clf = LinearRegression()
answer = cross_val_score(LR_clf, x, y2, scoring = "neg_mean_squared_error", cv = 10)
-answer

array([ 0.02983448,  0.02048526,  0.02735939,  0.02320408,  0.01944074,
        0.01262621,  0.01844842,  0.01222962,  0.02183042,  0.01548108])

#### ii. MSE on entrie set

In [15]:
answer = LR_clf.fit(x, y2)
y2_predict = LR_clf.predict(x)
print(mean_squared_error(y2, y2_predict))

0.0165167748803


#### iii. Most Precitive Featrues

For high crime rate

In [16]:
nvs = zip(features[3:-1], answer.coef_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('PersPerOccupHous', 0.6350881164986133),
 ('PctHousOwnOcc', 0.56813320988653082),
 ('MalePctDivorce', 0.45851704864166098),
 ('PctRecImmig8', 0.43251055765291008),
 ('MedRent', 0.37272779771138975),
 ('medFamInc', 0.28797887494607072),
 ('PctEmploy', 0.24847431608470519),
 ('MalePctNevMarr', 0.22672791284180535),
 ('PctPersDenseHous', 0.21435256414197748),
 ('OwnOccMedVal', 0.21287586365359706)]

For low crime rate

In [17]:
neg_sortFeature = sorted(nvDict.items(), key=lambda item:item[1])
neg_sortFeature[0:10]

[('PctPersOwnOccup', -0.67569447880286471),
 ('TotalPctDiv', -0.56192431441465263),
 ('whitePerCap', -0.3510157744407798),
 ('PctKids2Par', -0.32265127649627445),
 ('OwnOccLowQuart', -0.30817021919318288),
 ('numbUrban', -0.2964425431775653),
 ('PersPerRentOccHous', -0.25457166271646947),
 ('RentLowQ', -0.23475151630251967),
 ('agePct12t29', -0.22921783699885442),
 ('PctRecImmig5', -0.2182210073546518)]

### b. Use Ridge

#### i. MSE under 10-fold CV

In [4]:
Ri_clf = Ridge()
answer = cross_val_score(Ri_clf, x, y2, scoring = "neg_mean_squared_error", cv = 10)
-answer

array([ 0.0295138 ,  0.01972075,  0.02795035,  0.02311508,  0.01872138,
        0.0120445 ,  0.01780565,  0.01212225,  0.02191791,  0.01503854])

#### ii. MSE on the training set

In [5]:
Ri_clf = Ridge()
answer = Ri_clf.fit(x, y2)
y2_predict = Ri_clf.predict(x)
print(mean_squared_error(y2, y2_predict))

0.0167635291552


#### iii. best alpha

In [6]:
Rcv_clf = RidgeCV(alphas = (10, 1, 0.1, 0.01, 0.001), cv =10)
answer = Rcv_clf.fit(x, y2)
print(Rcv_clf.alpha_)

1


#### iv. About Overfitting

The ridge get little better result on 10-fold CV, but get little worse result on the training set, which means ridge takes less overfitting since it can predict more precisely on other test data while predict worese on training set itself.

### c. Use Polynomial Features to do quadratic polynomial

#### i. MSE of 10-fold CV

In [16]:
poly = PolynomialFeatures(degree=2)
x_ = poly.fit_transform(x)
LR_clf = LinearRegression()
answer = cross_val_score(LR_clf, x_, y2, scoring = "neg_mean_squared_error", cv = 10)
-answer

array([ 0.15479511,  0.14987831,  0.19852503,  0.10211292,  0.07857856,
        0.14101072,  0.09207106,  0.07377483,  0.16920729,  0.13902769])

#### ii. MSE on training set

In [21]:
LR_clf.fit(x_, y2)
print(mean_squared_error(y2, LR_clf.predict(x_)))

1.69580219538e-28


#### iii. Whether better

Since the quadratic polynomial regression get higher MSE than that in linear regression in 10-fold CV, this quadratic polynomial regression get worse match than linear model.

Moreover, on the training set, the quadratic polynomial regression getting better error value should be the result of overfitting.

## 4. Dirty Data

Read in the data from full csv file.

In [34]:
f2 = open("communities-crime-full.csv", "r")

data2 = []
crimeRate2 = []
highCrime2 = []
features2 = f2.readline().strip().split(",")
line = f2.readline()
while line:
    line = line.replace(",?",",nan")
    tokens = line.strip().split(",")
    data2.append(tokens[5:-1])
    crimeRate2.append(float(tokens[-1]))
    if(float(tokens[-1]) > 0.1):
        highCrime2.append(True)
    else:
        highCrime2.append(False)
    line = f2.readline()
x_2 = np.array(data2)
y_2 = np.array(highCrime2)
y_2_2= np.array(crimeRate2)
f2.close()

Handle the missing values

In [37]:
imp = Imputer(missing_values="NaN", strategy='most_frequent', axis=0)  
x_imp = imp.fit_transform(x_2)

### a. CV results

In [42]:
DT_clf = tree.DecisionTreeClassifier()
answer = cross_val_score(DT_clf, x_imp, y_2, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.80597015  0.745       0.735       0.72864322  0.72864322  0.73366834
  0.83417085  0.77386935  0.73869347  0.73366834]
The average of accuracy: 0.755732693317


In [43]:
answer = cross_val_score(DT_clf, x_imp, y_2, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.83870968  0.808       0.79661017  0.80152672  0.76984127  0.808
  0.86290323  0.84482759  0.79661017  0.78125   ]
The average of precision: 0.810827881581


In [45]:
answer = cross_val_score(DT_clf, x_imp, y_2, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.80952381  0.832       0.784       0.824       0.76        0.752       0.832
  0.808       0.752       0.792     ]
The average of recall: 0.794552380952


All the CV results becomes better after adding the dirty data. Therefore, the feature with missing values can also provide the trace, which can contribute to the prediction.

## 5. Teamwork： 

### i. (1) Random Forest

#### Clean data

Accuracy, Precision, Recall

In [3]:
RF_clf = RandomForestClassifier()
answer = cross_val_score(RF_clf, x, y, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.775       0.85        0.83        0.8040201   0.67336683  0.68844221
  0.83417085  0.75879397  0.8040201   0.80904523]
The average of accuracy: 0.782685929648


In [4]:
answer = cross_val_score(RF_clf, x, y, scoring = "precision", cv = 10)
print(answer)
print("The average of presion:", sum(answer)/len(answer))

[ 0.78321678  0.8828125   0.88709677  0.88235294  0.88043478  0.84848485
  0.8359375   0.94680851  0.83206107  0.83898305]
The average of presion: 0.861818875987


In [5]:
answer = cross_val_score(RF_clf, x, y, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.936  0.936  0.92   0.768  0.656  0.648  0.848  0.704  0.904  0.8  ]
The average of recall: 0.812


Key Features

In [11]:
RF_clf.fit(x, y)
nvs = zip(features[3:-1], RF_clf.feature_importances_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('PctKids2Par', 0.076824674000833287),
 ('PctIlleg', 0.074654892720768015),
 ('racePctWhite', 0.056145440464856887),
 ('TotalPctDiv', 0.041481966405490414),
 ('PctPersDenseHous', 0.040571605932169609),
 ('racepctblack', 0.036009739412114203),
 ('NumIlleg', 0.033192552779043476),
 ('pctWPubAsst', 0.030247875037522153),
 ('MalePctDivorce', 0.025354629420964187),
 ('pctWInvInc', 0.019293913825466358)]

#### Dirty data

Accuracy, Precision, Recall

In [74]:
RF_clf = RandomForestClassifier()
answer = cross_val_score(RF_clf, x_imp, y_2, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.84079602  0.83        0.855       0.80904523  0.78894472  0.8241206
  0.81407035  0.79396985  0.83417085  0.79899497]
The average of accuracy: 0.818911260282


In [75]:
answer = cross_val_score(RF_clf, x_imp, y_2, scoring = "precision", cv = 10)
print(answer)
print("The average of presion:", sum(answer)/len(answer))

[ 0.89344262  0.84677419  0.84920635  0.83333333  0.89915966  0.86956522
  0.8907563   0.91891892  0.83870968  0.86178862]
The average of presion: 0.870165489704


In [77]:
answer = cross_val_score(RF_clf, x_imp, y_2, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.84920635  0.896       0.848       0.848       0.824       0.808       0.8
  0.76        0.88        0.816     ]
The average of recall: 0.832920634921


Key Features

In [78]:
RF_clf.fit(x_imp, y_2)
nvs = zip(features[3:-1], RF_clf.feature_importances_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('PctPersDenseHous', 0.072821299288081259),
 ('PctKids2Par', 0.064593937780324767),
 ('NumIlleg', 0.055860359869965091),
 ('pctWPubAsst', 0.047493632627543518),
 ('FemalePctDiv', 0.046138502050214643),
 ('MalePctDivorce', 0.04309545725618473),
 ('PctIlleg', 0.038893994558554225),
 ('racePctWhite', 0.028212163050568072),
 ('PctHousLess3BR', 0.025966555330106632),
 ('PctNotHSGrad', 0.018593830982340638)]

### i. (2) GradientBoosting

#### Clean data

Accuracy, Precision, Recall

In [24]:
B_clf = GradientBoostingClassifier()
answer = cross_val_score(B_clf, x, y, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.79        0.89        0.88        0.81407035  0.69346734  0.72864322
  0.78894472  0.81407035  0.7638191   0.82914573]
The average of accuracy: 0.799216080402


In [25]:
answer = cross_val_score(B_clf, x, y, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.77124183  0.89230769  0.87407407  0.85365854  0.86363636  0.85858586
  0.81679389  0.94        0.75163399  0.8699187 ]
The average of precision: 0.84918509345


In [26]:
answer = cross_val_score(B_clf, x, y, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.944  0.92   0.944  0.832  0.6    0.68   0.856  0.752  0.92   0.856]
The average of recall: 0.8304


Key Featrues

In [27]:
B_clf.fit(x, y)
nvs = zip(features[3:-1], B_clf.feature_importances_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('PctKids2Par', 0.080401108416970929),
 ('racePctWhite', 0.060479605405120103),
 ('pctWInvInc', 0.027985251341367326),
 ('FemalePctDiv', 0.027479502008895925),
 ('PctIlleg', 0.024581419243101442),
 ('NumIlleg', 0.022599223727140544),
 ('PctEmplManu', 0.021020623390471095),
 ('PctWOFullPlumb', 0.02064146481338585),
 ('PctBornSameState', 0.020507724005285222),
 ('blackPerCap', 0.020210996418640548)]

#### Dirty data

Accuracy, Precision, Recall

In [79]:
B_clf = GradientBoostingClassifier()
answer = cross_val_score(B_clf, x_imp, y_2, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.85074627  0.83        0.85        0.81407035  0.83417085  0.82914573
  0.85427136  0.83919598  0.82914573  0.8241206 ]
The average of accuracy: 0.835486687167


In [80]:
answer = cross_val_score(B_clf, x_imp, y_2, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.88709677  0.85826772  0.8671875   0.83969466  0.8907563   0.864
  0.90677966  0.8974359   0.87096774  0.8515625 ]
The average of precision: 0.873374875013


In [81]:
answer = cross_val_score(B_clf, x_imp, y_2, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.87301587  0.864       0.896       0.872       0.848       0.864       0.856
  0.848       0.864       0.872     ]
The average of recall: 0.865701587302


Key Featrues

In [82]:
B_clf.fit(x_imp, y_2)
nvs = zip(features[3:-1], B_clf.feature_importances_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('PctKids2Par', 0.081483172806808127),
 ('racePctWhite', 0.052435582657868317),
 ('FemalePctDiv', 0.032857604141464838),
 ('PctImmigRec10', 0.024810590334708896),
 ('PctBornSameState', 0.023418099001997431),
 ('NumIlleg', 0.022179261853530217),
 ('pctWInvInc', 0.020899450840109957),
 ('PctImmigRec5', 0.020874866909534592),
 ('PctVacMore6Mos', 0.020319104558007593),
 ('HousVacant', 0.019388409742180999)]

### ii. Which method give best result

When we use the dirty data, we can know from the result that Gradient Boosting get the best accuracy, recall and Precision.
When we use the clean data, the Gradient Boosting get the best accuracy. 
It shows that the combination of base learners can really improve the result a lot.

Sepcifically, the GaussianNB get the best precision ,and the LinearSVC get best recall.

### iii. Key features

The featrue "PctKids2Par" seen to be the most consistency predictive of high crime rate. This conclusion should has high reliability since it always have a high weight among these methods.

## 6. Extra Work: (1) ada Boosting

### Clean data

Accuuracy, Precision, Recall

In [17]:
B_clf = AdaBoostClassifier()
answer = cross_val_score(B_clf, x, y, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.8         0.825       0.865       0.79396985  0.69849246  0.69849246
  0.80904523  0.77386935  0.76884422  0.79396985]
The average of accuracy: 0.782668341709


In [18]:
answer = cross_val_score(B_clf, x, y, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.77419355  0.82608696  0.88888889  0.828125    0.84946237  0.81553398
  0.80851064  0.96511628  0.77622378  0.83870968]
The average of precision: 0.837085111098


In [19]:
answer = cross_val_score(B_clf, x, y, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.96   0.912  0.896  0.848  0.632  0.672  0.912  0.664  0.888  0.832]
The average of recall: 0.8216


Key Features

In [20]:
B_clf.fit(x, y)
nvs = zip(features[3:-1], B_clf.feature_importances_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('racePctWhite', 0.12),
 ('PctKids2Par', 0.10000000000000001),
 ('TotalPctDiv', 0.059999999999999998),
 ('PctHousNoPhone', 0.059999999999999998),
 ('agePct12t21', 0.040000000000000001),
 ('pctWInvInc', 0.040000000000000001),
 ('PctTeen2Par', 0.040000000000000001),
 ('LandArea', 0.040000000000000001),
 ('population', 0.02),
 ('householdsize', 0.02)]

### Dirty data

Accuuracy, Precision, Recall

In [6]:
B_clf = AdaBoostClassifier()
answer = cross_val_score(B_clf, x_imp, y_2, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.84079602  0.85        0.775       0.81407035  0.8040201   0.81407035
  0.84924623  0.82914573  0.79396985  0.81909548]
The average of accuracy: 0.818941411035


In [7]:
answer = cross_val_score(B_clf, x_imp, y_2, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.86153846  0.88        0.82258065  0.81884058  0.84677419  0.86065574
  0.88617886  0.88888889  0.83870968  0.83969466]
The average of precision: 0.854386170225


In [8]:
answer = cross_val_score(B_clf, x_imp, y_2, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.88888889  0.88        0.816       0.904       0.84        0.84        0.872
  0.832       0.832       0.88      ]
The average of recall: 0.858488888889


Key Features

In [9]:
B_clf.fit(x_imp, y_2)
nvs = zip(features[3:-1], B_clf.feature_importances_)
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('racePctWhite', 0.10000000000000001),
 ('PctKids2Par', 0.10000000000000001),
 ('pctWInvInc', 0.059999999999999998),
 ('TotalPctDiv', 0.059999999999999998),
 ('agePct12t21', 0.040000000000000001),
 ('perCapInc', 0.040000000000000001),
 ('PctTeen2Par', 0.040000000000000001),
 ('population', 0.02),
 ('householdsize', 0.02),
 ('racepctblack', 0.02)]

## 6.Extra work: (2) SGDClassifier

### Clean data

Accuracy, Precision, Recall

In [13]:
SGD_clf = SGDClassifier()
answer = cross_val_score(SGD_clf, x, y, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.785       0.89        0.9         0.81407035  0.60301508  0.56281407
  0.8241206   0.85929648  0.79396985  0.79396985]
The average of accuracy: 0.782625628141


In [14]:
answer = cross_val_score(SGD_clf, x, y, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.82539683  0.73652695  0.95098039  0.98780488  0.89156627  0.83838384
  0.79861111  0.79054054  0.64921466  0.97560976]
The average of precision: 0.844463521259


In [16]:
answer = cross_val_score(SGD_clf, x, y, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.88   0.88   0.936  0.84   0.984  0.648  0.976  0.496  0.984  0.608]
The average of recall: 0.8232


Key feature

In [11]:
SGD_clf.fit(x,y)
nvs = zip(features[3:-1], SGD_clf.coef_[0])
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('racepctblack', 18.515140459686272),
 ('PctIlleg', 10.525355709595019),
 ('racePctHisp', 10.142283838015292),
 ('MalePctDivorce', 9.2666909886902182),
 ('OtherPerCap', 9.2484494709959915),
 ('TotalPctDiv', 9.2028456767602904),
 ('FemalePctDiv', 8.2269244801166952),
 ('PctPersDenseHous', 8.2178037212695774),
 ('LemasPctOfficDrugUn', 7.415176942721625),
 ('MedRentPctHousInc', 7.3148485954031219)]

In [12]:
neg_sortFeature = sorted(nvDict.items(), key=lambda item:item[1])
neg_sortFeature[0:10]

[('racePctWhite', -18.934695366654498),
 ('pctWInvInc', -18.515140459686219),
 ('PctKids2Par', -11.255016417365889),
 ('PctFam2Par', -8.3546151039766254),
 ('PctHousOccup', -8.0353885443268585),
 ('MedOwnCostPctIncNoMtg', -7.3330901130974278),
 ('RentLowQ', -6.2112367748996817),
 ('PctWOFullPlumb', -6.2021160160525648),
 ('PctYoungKids2Par', -6.0926669098868711),
 ('PctBSorMore', -5.1623495074790506)]

### Dirty data

Accuracy, Precision, Recall

In [65]:
SGD_clf = SGDClassifier()
answer = cross_val_score(SGD_clf, x_imp, y_2, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.82089552  0.805       0.75        0.83919598  0.7839196   0.70351759
  0.69346734  0.8241206   0.82914573  0.81407035]
The average of accuracy: 0.786333270832


In [18]:
answer = cross_val_score(SGD_clf, x_imp, y_2, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.72352941  0.7852349   0.79020979  0.67934783  0.832       0.93269231
  0.74390244  0.94230769  0.98765432  0.87272727]
The average of precision: 0.828960596013


In [19]:
answer = cross_val_score(SGD_clf, x_imp, y_2, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.65079365  1.          0.968       0.76        0.584       0.88        0.672
  0.4         0.976       0.824     ]
The average of recall: 0.771479365079


Key feature

In [20]:
SGD_clf.fit(x_imp,y_2)
nvs = zip(features[3:-1], SGD_clf.coef_[0])
nvDict = dict((name,value) for name,value in nvs)
sortFeature = sorted(nvDict.items(), key=lambda item:-item[1])
sortFeature[0:10]

[('racepctblack', 20.594402406782777),
 ('PctIlleg', 11.477801075758917),
 ('MalePctDivorce', 10.274409700063803),
 ('racePctHisp', 8.4966724405141285),
 ('TotalPctDiv', 8.4510894338590319),
 ('PctPersDenseHous', 7.1291822408606009),
 ('PctImmigRec10', 6.3269213237305051),
 ('FemalePctDiv', 6.1445892971100111),
 ('OtherPerCap', 6.135472695779014),
 ('HousVacant', 5.7434588385449894)]

In [21]:
neg_sortFeature = sorted(nvDict.items(), key=lambda item:item[1])
neg_sortFeature[0:10]

[('racePctWhite', -20.667335217430931),
 ('pctWInvInc', -19.190445801805044),
 ('PctKids2Par', -11.669249703710429),
 ('PctWOFullPlumb', -8.3872732245419392),
 ('RentLowQ', -8.1411249886042487),
 ('PctFam2Par', -8.0408423739629828),
 ('PctHousOccup', -7.2568146594949239),
 ('HispPerCap', -5.9349074664964885),
 ('PctBSorMore', -5.907557662503403),
 ('PctOccupManu', -5.0870635427112987)]

### i. Which method give best result

The situation is similar with that in team work, but this time the SGD get best Precision.

### ii. Best Feature

The featrue "PctKids2Par" seen to be the most consistency predictive of high crime rate. This conclusion should has high reliability since it always have a high weight among these methods.

## 6.Extra work: team work for three people

### ii. The most useful threshold 

In [14]:
crimeT = []
for attr in crimeRate:
    tuple = []
    tuple.append(attr)
    crimeT.append(tuple) 
K_clf = KMeans(n_clusters=2)
K_clf.fit(crimeT)
K_clf.cluster_centers_

array([[ 0.13872487],
       [ 0.63204489]])

In [74]:
threshold = abs(K_clf.cluster_centers_[0][0] + K_clf.cluster_centers_[1][0])/2
print(threshold)

0.385413529892


The threshold can be 0.3854 for clean data. It is useful because it divides the crime rates into two relative compacted and different clusters.

In [39]:
crimeT2 = []
for attr in crimeRate2:
    tuple = []
    tuple.append(attr)
    crimeT2.append(tuple) 
K_clf = KMeans(n_clusters=2)
K_clf.fit(crimeT2)
K_clf.cluster_centers_

array([[ 0.13878217],
       [ 0.63204489]])

In [75]:
threshold2 = abs(K_clf.cluster_centers_[0][0] + K_clf.cluster_centers_[1][0])/2
print(threshold2)

0.385413529892


For dirty data, the useful threshold can be 0.3854.

### iii. the new CV

Caculate the new lable by the new threshold

In [76]:
newCrime = []
for attr in crimeRate:
    if(attr > threshold):
        newCrime.append(True)
    else:
        newCrime.append(False)
y_new = np.array(newCrime)

newCrime_full = []
for attr in crimeRate2:
    if(attr > threshold2):
        newCrime_full.append(True)
    else:
        newCrime_full.append(False)
y_2_new = np.array(newCrime_full)

#### (1) decision tree

The CV results for clean data

In [77]:
Tree_clf = tree.DecisionTreeClassifier()
answer = cross_val_score(Tree_clf, x, y_new, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.76119403  0.835       0.69849246  0.81407035  0.84422111  0.77889447
  0.85427136  0.85929648  0.64824121  0.77386935]
The average of accuracy: 0.786755081377


In [78]:
answer = cross_val_score(Tree_clf, x, y_new, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.42        0.6         0.34920635  0.53125     0.55813953  0.44444444
  0.65116279  0.61111111  0.32098765  0.38461538]
The average of precision: 0.487091726928


In [79]:
answer = cross_val_score(Tree_clf, x, y_new, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.63414634  0.55        0.525       0.5         0.5         0.575       0.65
  0.65        0.65        0.425     ]
The average of recall: 0.565914634146


The CV results for Dirty data

In [80]:
answer = cross_val_score(Tree_clf, x_imp, y_2_new, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.8159204   0.8         0.81        0.84924623  0.83919598  0.79899497
  0.83417085  0.79899497  0.85929648  0.84924623]
The average of accuracy: 0.825506612665


In [81]:
answer = cross_val_score(Tree_clf, x_imp, y_2_new, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.52631579  0.43181818  0.5         0.55813953  0.66666667  0.5
  0.61363636  0.47727273  0.64285714  0.64864865]
The average of precision: 0.556535505526


In [82]:
answer = cross_val_score(Tree_clf, x_imp, y_2_new, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.48780488  0.5         0.7         0.725       0.525       0.55        0.625
  0.475       0.725       0.6       ]
The average of recall: 0.591280487805


#### (2) Gradient Boosting

The CV results for clean data

In [83]:
B_clf = GradientBoostingClassifier()
answer = cross_val_score(B_clf, x, y_new, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.82587065  0.89        0.78894472  0.89949749  0.87437186  0.84924623
  0.89949749  0.89949749  0.72864322  0.83417085]
The average of accuracy: 0.84897399935


In [84]:
answer = cross_val_score(B_clf, x, y_new, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.575       0.76470588  0.47727273  0.77777778  0.72727273  0.65625     0.8
  0.83333333  0.4         0.6       ]
The average of precision: 0.661161244801


In [85]:
answer = cross_val_score(B_clf, x, y_new, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.56097561  0.625       0.525       0.7         0.6         0.525       0.7
  0.625       0.7         0.45      ]
The average of recall: 0.601097560976


The CV results for dirty data

In [86]:
answer = cross_val_score(B_clf, x_imp, y_2_new, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.88557214  0.89        0.87        0.88944724  0.89949749  0.90954774
  0.90954774  0.87939698  0.90954774  0.87437186]
The average of accuracy: 0.891692892322


In [87]:
answer = cross_val_score(B_clf, x_imp, y_2_new, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.79310345  0.75        0.64444444  0.73684211  0.77142857  0.78947368
  0.84375     0.73529412  0.80555556  0.71052632]
The average of precision: 0.758041824261


In [89]:
answer = cross_val_score(B_clf, x_imp, y_2_new, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.58536585  0.675       0.725       0.7         0.7         0.75        0.675
  0.625       0.725       0.675     ]
The average of recall: 0.683536585366


#### (3) GaussianNB

The CV results for clean data

In [90]:
G_clf = GaussianNB()
answer = cross_val_score(G_clf, x, y_new, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.75124378  0.825       0.84422111  0.88442211  0.85427136  0.83919598
  0.87437186  0.87939698  0.72361809  0.83417085]
The average of accuracy: 0.83099121228


In [91]:
answer = cross_val_score(G_clf, x, y_new, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.44        0.54237288  0.57142857  0.71794872  0.62222222  0.6
  0.63636364  0.69047619  0.41176471  0.57142857]
The average of precision: 0.580400549711


In [92]:
answer = cross_val_score(G_clf, x, y_new, scoring = "recall", cv = 10)
print(answer)
print("The average of recall:", sum(answer)/len(answer))

[ 0.80487805  0.8         0.9         0.7         0.7         0.6         0.875
  0.725       0.875       0.7       ]
The average of recall: 0.767987804878


The CV results for clean data

In [93]:
answer = cross_val_score(G_clf, x_imp, y_2_new, scoring = "accuracy", cv = 10)
print(answer)
print("The average of accuracy:", sum(answer)/len(answer))

[ 0.81094527  0.83        0.82        0.83919598  0.85427136  0.85929648
  0.84924623  0.81909548  0.83417085  0.85929648]
The average of accuracy: 0.837551813795


In [94]:
answer = cross_val_score(G_clf, x_imp, y_2_new, scoring = "precision", cv = 10)
print(answer)
print("The average of precision:", sum(answer)/len(answer))

[ 0.53488372  0.58333333  0.54761905  0.57692308  0.66666667  0.65789474
  0.64705882  0.55882353  0.58536585  0.65789474]
The average of precision: 0.601646352576


In [95]:
answer = cross_val_score(G_clf, x_imp, y_2_new, scoring = "recall", cv = 10)
print(answer)
print("The average of reacall:", sum(answer)/len(answer))

[ 0.56097561  0.525       0.575       0.75        0.55        0.625       0.55
  0.475       0.6         0.625     ]
The average of reacall: 0.583597560976


### iv. The difference and the enlightenment

Using the new threshold, all the accuracy of these classifiers are improved, while the precision and recall are relative worse. This shows that clustering (or labling) the output in a better way can improve the performance of prediction since the accuracy is better. Additionally, the precision and recall becoming worse is just showing that the prediction on postive data is worse.(But the total accuracy is increasing)

This situation tells us that we should lable the output in a better way which is dividing them into relative seperated and compacted clusters. Under this way, we can get high accuracy using classifier.